# Introduction
The following examples show how to use the **intand** (**INT**eractive **AN**omaly **D**etection) to detect anomalies in a dataset by repeatedly interacting with a human expert and incorporating his feedback.

Installation:

```
$ git clone XXX
$ cd interactive-anomaly-detection
$ pip install .
```

In [4]:
import os
%matplotlib notebook
import matplotlib.pyplot as plt
import sys
sys.path
sys.path.append('/Users/chenglingyun/Desktop/interactive-anomaly-detection/examples')
from intand import datasets, Evaluation, feedback_loop
from intand.ensembles import IsoForest
from intand.models import Unsupervised, OJRank, OMD, AAD,PAD
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import math
import intand
print("VERSION:", intand.__version__)

VERSION: 0.1.0


In [ ]:
#path where data is stored
path = '/Users/chenglingyun/Desktop/interactive-anomaly-detection/examples/intand/datasets/data'

files = os.listdir(path)
for data in files:
    dataset=data.split('.')[0]
    #loading dataset
    X, y = datasets.load_dataset(dataset, normalize=True)
    dim=X.shape
    n = dim[0]
    root = int(math.sqrt(n))
    
    name='myExperiment-LOF-root-'+dataset
    eva = Evaluation(y, expname=name)
    for _ in range(5):
        #PAD
        '''Z1 = IsoForest(n_trees=1).fit(X).transform(X)
        My_pred=np.full((X.shape[0], 1), -1)
        anomaly_ind=np.argsort(Z1,axis=0)[-(int (0.1*X.shape[0])):]
        My_pred[anomaly_ind]=1
        pseudoy = np.ravel(My_pred ,order='C')'''
        
        lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
        y_pred = lof.fit_predict(X)
        pseudoy = -y_pred
        clf = RandomForestClassifier(n_estimators=50, random_state=1)
        scores = clf.fit(X, pseudoy ).predict_proba(X)[:, 1]
        Zp = np.asmatrix(scores)
        Zp = np.transpose(Zp)
        
    
        #Z with cluster
        isolation_forest = IsolationForest(n_estimators=100,contamination='auto',behaviour='deprecated').fit(X)
        anomaly_score = -(isolation_forest.score_samples(X))
        Z_cluster=np.array([], dtype=np.int64).reshape(dim[0],0)
        for cluster in range(2,11):
            kmeans = KMeans(n_clusters=cluster).fit(X).labels_
            z=np.zeros((dim[0], cluster))
            for instance in range(dim[0]):
                col=kmeans[instance]
                z[instance,col]=anomaly_score[instance]
            Z_cluster=np.hstack([Z_cluster,z])
        
        #Z original
        Z = IsoForest().fit(X).transform(X)
        eva.run([Unsupervised(Z, X), OJRank(Z, X), OMD(Z, X),Unsupervised(Z_cluster, X), OJRank(Z_cluster, X), OMD(Z_cluster, X), PAD(Zp,pseudoy,root,X) ])
    name_g=name+'.json'
    eva.plot_results(name_g, "precision")
    eva.plot_results(name_g, "effort")    